In [4]:

from nltk.corpus import stopwords
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from utils import clean_str, loadWord2Vec

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)



{'when', 'o', 'me', 'hadn', "she's", 'i', 'she', 'itself', 'm', 'the', 'shan', "doesn't", 'to', 'they', 'having', 'who', 'being', 'against', "shan't", 'don', 'here', 's', 'hasn', 'while', 'as', 'after', 'this', 'weren', 'will', 'my', 'whom', 'didn', 'until', 'mightn', "wasn't", 'himself', 'more', 'needn', 'couldn', 'ourselves', 'such', 'yourself', 'because', "that'll", 'won', 'of', 'any', 'both', 'am', 'an', 'on', 'was', 'each', 'some', "you'd", 'nor', 'not', 've', "needn't", 'before', 'same', 'now', 'wouldn', 'were', 'for', 'has', 'in', 'between', "wouldn't", 'his', 'mustn', 'very', 're', 'herself', 'other', 'again', 'yourselves', 'or', 'are', 'below', 'ours', 'do', 'hers', 'its', 'wasn', "weren't", 'be', 'yours', 'where', "should've", "mightn't", 'and', "don't", 'll', 'y', 'so', 'been', "it's", "won't", 'then', 'few', 'just', "hadn't", 'from', 'him', 'if', 'you', "aren't", 'how', "shouldn't", 'what', 'by', 'those', 'is', 'with', 'only', 'themselves', 'them', 'had', 'doesn', 'about', 

[nltk_data] Downloading package stopwords to /home/sjain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import random

doc_content_list = []
f = open('data/corpus/all_corpus.txt', 'rb')
# f = open('data/wiki_long_abstracts_en_text.txt', 'r')
for line in f.readlines():
    doc_content_list.append(line.strip().decode('latin1'))
f.close()
total_lines = len(doc_content_list)

# Create a list of indices (0 to total_lines-1)
line_indices = list(range(total_lines))

# Randomly select 100 indices from the line indices
selected_indices = random.sample(line_indices, min(1500, total_lines))

# Retrieve the corresponding document contents using the selected indices
random_lines = [doc_content_list[i] for i in selected_indices]
doc_content_list = random_lines
print(selected_indices)
print(doc_content_list[:5])

[30678, 110533, 142793, 72556, 27639, 94748, 18254, 169291, 47676, 160940, 147744, 15127, 78749, 59829, 167317, 73056, 131571, 24531, 118370, 148336, 76346, 165969, 30699, 114240, 32867, 120188, 44423, 106717, 19155, 132223, 12769, 23208, 70517, 75990, 65828, 101146, 27107, 70210, 4574, 89551, 27173, 49976, 149660, 1760, 69609, 49318, 124182, 94595, 123304, 105721, 86350, 159541, 79606, 123028, 107095, 150414, 4310, 64724, 7650, 126763, 135611, 65948, 114278, 94247, 142386, 168140, 128973, 121514, 130405, 34315, 143244, 3719, 61959, 112651, 38171, 148259, 113456, 112232, 73525, 7920, 36853, 116489, 84397, 136277, 123428, 76576, 5151, 56941, 124699, 12533, 18178, 19649, 100381, 78313, 70090, 155329, 135682, 53481, 151208, 130240, 27646, 28675, 79973, 157667, 43100, 53963, 78501, 166603, 151739, 110517, 35089, 132575, 138224, 59489, 90720, 61597, 78225, 10642, 107487, 140943, 126687, 104644, 942, 132278, 85628, 160146, 161788, 39811, 37744, 48316, 985, 152908, 122448, 69097, 16642, 96790

In [10]:
word_freq = {}  # to remove rare words

for doc_content in doc_content_list:
    temp = clean_str(doc_content)
    words = temp.split()
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1
words

['comprehensive',
 'review',
 'of',
 'audio',
 'steganalysis',
 'methods',
 'recently',
 ',',
 'merging',
 'signal',
 'processing',
 'techniques',
 'with',
 'information',
 'security',
 'services',
 'has',
 'drawn',
 'a',
 'lot',
 'of',
 'attention',
 'steganography',
 'and',
 'steganalysis',
 'are',
 'among',
 'those',
 'trends',
 'like',
 'their',
 'counterparts',
 'in',
 'cryptology',
 ',',
 'steganography',
 'and',
 'steganalysis',
 'are',
 'in',
 'a',
 'constant',
 'battle',
 'steganography',
 'methods',
 'try',
 'to',
 'hide',
 'the',
 'presence',
 'of',
 'covert',
 'messages',
 'in',
 'innocuous',
 'looking',
 'data',
 ',',
 'whereas',
 'steganalysis',
 'methods',
 'try',
 'to',
 'break',
 'steganography',
 'algorithms',
 'and',
 'reveal',
 'the',
 'existence',
 'of',
 'hidden',
 'messages',
 'the',
 'streaming',
 'nature',
 'of',
 'audio',
 'signals',
 ',',
 'their',
 'popularity',
 ',',
 'and',
 'their',
 'wide',
 'spread',
 'usage',
 'make',
 'them',
 'very',
 'good',
 'candi

In [11]:


clean_docs = []
for doc_content in doc_content_list:
    temp = clean_str(doc_content)
    words = temp.split()
    doc_words = []
    for word in words:
        # word not in stop_words and word_freq[word] >= 5
        if word not in stop_words and word_freq[word] >= 5:
            doc_words.append(word)

    doc_str = ' '.join(doc_words).strip()
    #if doc_str == '':
        #doc_str = temp
    clean_docs.append(doc_str)
clean_docs[:5]

['late weak markov automata weak bisimilarity distribution based equivalence notion markov automata gained popularity reasonable behavioural equivalence markov automata paper studies strictly notion late weak bisimilarity enjoys valuable properties important subclasses trace distribution equivalence partial information , compositionality preserved distributed intersection two scheduler classes thus still reasonable compositional theory markov automata',
 'interactive segmentation medical images fully convolutional neural networks image segmentation plays essential role medicine tasks segmentation approaches either manual , semi automated fully automated manual segmentation offers full control quality results , , time consuming prone operator bias fully automated methods require human effort , often deliver sub optimal results without providing users means make corrections semi automated approaches keep users control results providing means interaction , main challenge offer good trade 

In [12]:

clean_corpus_str = '\n'.join(clean_docs)

f = open('data/corpus/clean_1500.txt', 'w')
#f = open('data/wiki_long_abstracts_en_text.clean.txt', 'w')
f.write(clean_corpus_str)
f.close()

#dataset = '20ng'
# min_len = 10000
# aver_len = 0
# max_len = 0 

# f = open('data/corpus/' + dataset + '.clean.txt', 'r')
# #f = open('data/wiki_long_abstracts_en_text.txt', 'r')
# lines = f.readlines()
# for line in lines:
#     line = line.strip()
#     temp = line.split()
#     aver_len = aver_len + len(temp)
#     if len(temp) < min_len:
#         min_len = len(temp)
#     if len(temp) > max_len:
#         max_len = len(temp)
# f.close()
# aver_len = 1.0 * aver_len / len(lines)
# print('min_len : ' + str(min_len))
# print('max_len : ' + str(max_len))
# print('average_len : ' + str(aver_len))